In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!pip install pyspark
!pip install findspark
!pip install tensorflowonspark
!pip install sparkdl
!pip install tensorframes
!pip install kafka
!apt-get install openjdk-8-jdk-headless -qq > /dev/null\n
!wget -q  https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xvf spark-2.4.4-bin-hadoop2.7.tgz

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 665, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/cmdoptions.py", line 24, in <module>
    from pip._internal.models.search_scope import SearchScope
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/models/search_scope.py", line 11, in <module>
    from pip._internal.utils.misc import normalize_path, redact_password_from_url
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/utils/misc.py", line 21, in <module>
    from pip._vendor import pkg_resources
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3241, in <module>
    @_call_aside
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3225, in _call_aside
    f(*args

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.6"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages databricks:tensorframes:0.6.0-s_2.11 pyspark-shell'

In [0]:
!mkdir /root/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
cd /root/.kaggle

/root/.kaggle


In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"samuelpreethaml","key":"14617aaa25068d81fb4b1232097ec0ec"}'}

In [0]:
!mkdir /root/recommender

In [0]:
cd /root/recommender

/root/recommender


In [0]:
!kaggle competitions download -c santander-product-recommendation
!unzip train_ver2.csv.zip
!unzip test_ver2.csv.zip

sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_ver2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_ver2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  train_ver2.csv.zip
replace train_ver2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  test_ver2.csv.zip
replace test_ver2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: B
error:  invalid response [B]
replace test_ver2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
import findspark
from pyspark.sql.types import StructType, StructField, FloatType, StringType
import pyspark.sql.functions as F 
import sklearn
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
import pyspark.mllib
import keras
import tensorflowonspark
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorframes as tfs
from pyspark.sql import Row


Using TensorFlow backend.


In [0]:
data=pd.read_csv('train_ver2.csv','r',encoding='utf=8',delimiter=',')
headers=['Partitioned Date','Customer code','Employee index','Country residence','sex','Age','Date of First holder','New customer Index','Customer seniority','Primary Customer','Last date as primary customer','type at beginning','relation at beginning','Residence index','Foreigner index','Spouse index','channel to join','Deceased index','Addres type','Province code','Province name','Activity index','Gross income','segmentation','Saving Account','Guarantees','Current Accounts','Derivada Account','Payroll Account','Junior Account','Más particular Account','particular Account','particular Plus Account','Short-term deposits','Medium-term deposits','Long-term deposits','e-account','Funds','Mortgage','Pensions','Loans','Taxes','Credit Card','Securities','Home Account','Payroll','Pensions2','Direct Debit']
data.columns=headers

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
del(data['Employee index'])
del(data['Country residence'])
del(data['Date of First holder'])
del(data['Primary Customer'])
del(data['Last date as primary customer'])
del(data['type at beginning'])
del(data['Residence index'])
del(data['Spouse index'])
del(data['Deceased index'])
del(data['Addres type'])
del(data['Province name'])
data['Total Purchases']=data.iloc[:,-24:].sum(axis=1)
data

,Partitioned Date,Customer code,sex,Age,New customer Index,Customer seniority,relation at beginning,Foreigner index,channel to join,Province code,Activity index,Gross income,segmentation,Saving Account,Guarantees,Current Accounts,Derivada Account,Payroll Account,Junior Account,Más particular Account,particular Account,particular Plus Account,Short-term deposits,Medium-term deposits,Long-term deposits,e-account,Funds,Mortgage,Pensions,Loans,Taxes,Credit Card,Securities,Home Account,Payroll,Pensions2,Direct Debit,Total Purchases
0,2015-01-28,1375586,H,35,0.0,6,A,N,KHL,29.0,1.0,87218.10,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1.0
1,2015-01-28,1050611,V,23,0.0,35,I,S,KHE,13.0,0.0,35548.74,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1.0
2,2015-01-28,1050612,V,23,0.0,35,I,N,KHE,13.0,0.0,122179.11,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1.0
3,2015-01-28,1050613,H,22,0.0,35,I,N,KHD,50.0,0.0,119775.54,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1.0
4,2015-01-28,1050614,V,23,0.0,35,A,N,KHE,50.0,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1.0
5,2015-01-28,1050615,H,23,0.0,35,I,N,KHE,45.0,0.0,22220.04,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1.0
6,2015-01-28,1050616,H,23,0.0,35,I,N,KHE,24.0,0.0,295590.36,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1.0
7,2015-01-28,1050617,H,23,0.0,35,A,N,KHE,50.0,1.0,113316.66,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1.0
8,2015-01-28,1050619,H,24,0.0,35,I,N,KHE,20.0,0.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1.0
9,2015-01-28,1050620,H,23,0.0,35,I,N,KHE,10.0,0.0,113194.98,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,1.0


In [0]:
def label_encode(df, features, name):
    df[name] = df[name].astype('str')
    if name in transformers: # test
        df[name] = transformers[name].transform(df[name])
    else: # train
        transformers[name] = preprocessing.LabelEncoder()
        df[name] = transformers[name].fit_transform(df[name])
    features.append(name)
features=[]
transformers={}
data["Gross income"].fillna(1.0, inplace=True)
data["Customer seniority"]=data["Customer seniority"].replace('     NA',0)
data['Customer seniority']=data['Customer seniority'].map(lambda x :int(x))
data["Customer seniority"]=data["Customer seniority"].replace(-999999,0)
data['Customer seniority']=data['Customer seniority'].map(lambda x :int(x))
data['sex'].fillna(data['sex'].value_counts().index[0],inplace=True)
data['Age']=data['Age'].replace(' NA',data['Age'].value_counts().index[0])
data['Age']=data['Age'].map(lambda x :int(x))
data['relation at beginning'].fillna(data['relation at beginning'].value_counts().index[0],inplace=True)
data['channel to join'].fillna(data['channel to join'].value_counts().index[0],inplace=True)
data['Province code'].fillna(data['Province code'].value_counts().index[0],inplace=True)
data['Activity index'].fillna(data['Province code'].value_counts().index[0],inplace=True)
data['segmentation'].fillna(data['segmentation'].value_counts().index[0],inplace=True)
label_encode(data, features, "channel to join") # simple label encode
# label_encode(data, features, "nomprov") # use cod_prov only
label_encode(data, features, 'Activity index')
label_encode(data, features, "Foreigner index")
label_encode(data, features, "sex",)
label_encode(data, features, "New customer Index")
label_encode(data, features, "segmentation")
label_encode(data, features, "relation at beginning")
label_encode(data, features, "Customer code")
data["Partitioned Month"] = data["Partitioned Date"].map(lambda x: int(x.split("-")[1])).astype(np.int8)
data["Partitioned Year"] = data["Partitioned Date"].map(lambda x: int(x.split("-")[0])).astype(np.int64)
label_encode(data, features, "Partitioned Year")
del(data["Partitioned Date"])
data["Payroll"].fillna(0, inplace=True)
data['Pensions2'].fillna(0, inplace=True)
data.drop(data[data['Total Purchases'] == 0].index, inplace = True)

In [0]:
data=data[['Partitioned Month', 'Partitioned Year', "Customer code",'sex', 'Age', 'New customer Index','Customer seniority', 'relation at beginning', 'Foreigner index','channel to join', 'Province code', 'Activity index','Gross income','segmentation','Total Purchases','Saving Account', 'Guarantees', 'Current Accounts','Derivada Account', 'Payroll Account', 'Junior Account','Más particular Account', 'particular Account','particular Plus Account', 'Short-term deposits','Medium-term deposits', 'Long-term deposits', 'e-account', 'Funds','Mortgage', 'Pensions', 'Loans', 'Taxes', 'Credit Card', 'Securities','Home Account', 'Payroll', 'Pensions2', 'Direct Debit']]
train=data[:(11091070-696539)]
test=data[(11091070-696539):]
catf=['Partitioned Month','Partitioned Year',"Customer code",'sex','New customer Index', 'relation at beginning','Foreigner index',  'channel to join', 'Province code', 'Activity index','segmentation']
conf=[col for col in train.columns[0:15] if col not in catf]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(train.iloc[:,0:15], train.iloc[:,15:],test_size=0.25, random_state=42)
X_train

,Partitioned Month,Partitioned Year,Customer code,sex,Age,New customer Index,Customer seniority,relation at beginning,Foreigner index,channel to join,Province code,Activity index,Gross income,segmentation,Total Purchases
8774888,12,0,792706,0,43,0,96,0,0,120,28.0,1,124666.14,1,6.0
10998667,3,1,268789,1,23,0,18,1,0,150,13.0,0,35613.90,2,1.0
1353287,3,0,859172,1,44,0,62,0,0,120,28.0,1,113360.19,1,2.0
2938340,5,0,712774,0,46,0,221,0,0,24,28.0,1,179387.43,1,6.0
7297368,11,0,580509,1,92,0,165,1,0,122,10.0,0,51454.71,1,1.0
8402654,12,0,117042,0,22,0,29,0,0,150,8.0,1,155652.18,2,2.0
3186270,6,0,101621,1,44,0,30,0,0,122,28.0,1,112590.72,1,2.0
12692248,4,1,245146,1,21,0,20,1,0,150,15.0,0,76085.46,2,1.0
6630841,10,0,372458,1,23,1,2,1,0,158,46.0,1,1.00,2,1.0
8255141,12,0,848286,0,44,0,73,0,0,122,20.0,1,1.00,1,1.0


In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(24, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(24, activation=tf.nn.sigmoid)
])

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [0]:
model.fit(X_train.values, y_train.values, epochs=5)

Epoch 1/5
7795898/7795898 [==============================] - 727s 93us/sample - loss: 0.0399 - acc: 0.9997
Epoch 2/5
7795898/7795898 [==============================] - 725s 93us/sample - loss: 0.0028 - acc: 0.9999
Epoch 3/5
 536928/7795898 [=>............................] - ETA: 11:27 - loss: 0.0022 - acc: 0.9999

KeyboardInterrupt: ignored

In [0]:
model.evaluate(X_test, y_test)

2598633/2598633 [==============================] - 135s 52us/sample - loss: 0.0021 - acc: 0.9999


[0.002129851227199179, 0.99986804]

In [0]:
sc = SparkContext(master='local',appName='test1')

FileNotFoundError: ignored

In [0]:
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

NameError: ignored

In [0]:
data = [Row(i=int(x)) for x in range(10)]
df = spark.createDataFrame(data)
tf.reset_default_graph()
df.show()
x=tfs.row(df,'i')

W0905 09:48:17.428291 140342565549952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorframes/core.py:453: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



+---+
|  i|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [0]:
x.shape

TensorShape([])

In [0]:
!wget http://apachemirror.wuchna.com/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz

--2019-09-05 07:17:02--  http://apachemirror.wuchna.com/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
Resolving apachemirror.wuchna.com (apachemirror.wuchna.com)... 159.65.154.237
Connecting to apachemirror.wuchna.com (apachemirror.wuchna.com)|159.65.154.237|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9136463 (8.7M) [application/x-gzip]
Saving to: ‘apache-maven-3.6.1-bin.tar.gz’

apache-maven-3.6.1- 100%[===================>]   8.71M  3.41MB/s    in 2.6s    

2019-09-05 07:17:05 (3.41 MB/s) - ‘apache-maven-3.6.1-bin.tar.gz’ saved [9136463/9136463]



In [0]:
!sudo tar -xvzf apache-maven-3.6.1-bin.tar.gz

apache-maven-3.6.1/README.txt
apache-maven-3.6.1/LICENSE
apache-maven-3.6.1/NOTICE
apache-maven-3.6.1/lib/
apache-maven-3.6.1/lib/slf4j-api.license
apache-maven-3.6.1/lib/checker-compat-qual.license
apache-maven-3.6.1/lib/jsr250-api.license
apache-maven-3.6.1/lib/jcl-over-slf4j.license
apache-maven-3.6.1/lib/org.eclipse.sisu.plexus.license
apache-maven-3.6.1/lib/animal-sniffer-annotations.license
apache-maven-3.6.1/lib/org.eclipse.sisu.inject.license
apache-maven-3.6.1/lib/jansi-native/
apache-maven-3.6.1/lib/jansi-native/freebsd64/
apache-maven-3.6.1/lib/jansi-native/osx/
apache-maven-3.6.1/lib/jansi-native/windows32/
apache-maven-3.6.1/lib/jansi-native/linux32/
apache-maven-3.6.1/lib/jansi-native/freebsd32/
apache-maven-3.6.1/lib/jansi-native/windows64/
apache-maven-3.6.1/lib/jansi-native/linux64/
apache-maven-3.6.1/lib/jansi-native/freebsd64/libjansi.so
apache-maven-3.6.1/lib/jansi-native/osx/libjansi.jnilib
apache-maven-3.6.1/lib/jansi-native/windows32/jansi.dll
apache-maven-3.6.1/

In [0]:
!echo 'M2_HOME="/opt/apache-maven-3.6.1"'>environment

In [0]:
!echo 'PATH="/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/jvm/jdk-10.0.2/bin:/opt/apache-maven-3.6.0/bin"'>>environment

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.6"
os.environ["PATH"] = "/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/jvm/jdk-10.0.2/bin:/opt/apache-maven-3.6.0/bin"
os.environ["M2_HOME"] = "/opt/apache-maven-3.6.1"

In [0]:
!echo 'JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"'>>environment
!echo 'SPARK_HOME = "/content/spark-2.4.3-bin-hadoop2.6"'>>environment

In [0]:
!sudo update-alternatives --install "/usr/bin/mvn" "mvn" "/opt/apache-maven-3.6.1/bin/mvn" 0


In [0]:
!sudo update-alternatives --set mvn /opt/apache-maven-3.6.1/bin/mvn

In [0]:
!sudo wget https://raw.github.com/dimaj/maven-bash-completion/master/bash_completion.bash --output-document /etc/bash_completion.d/mvn

--2019-09-05 08:45:49--  https://raw.github.com/dimaj/maven-bash-completion/master/bash_completion.bash
Resolving raw.github.com (raw.github.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.github.com (raw.github.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/dimaj/maven-bash-completion/master/bash_completion.bash [following]
--2019-09-05 08:45:50--  https://raw.githubusercontent.com/dimaj/maven-bash-completion/master/bash_completion.bash
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15780 (15K) [text/plain]
Saving to: ‘/etc/bash_completion.d/mvn’

/etc/bash_completio 100%[===================>]  15.41K  --.-KB/s    in 0.01s   

2019-09-0

In [0]:
!mvn -e

[INFO] Error stacktraces are turned on.
[INFO] Scanning for projects...
[INFO] ------------------------------------------------------------------------
[INFO] BUILD FAILURE
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  0.096 s
[INFO] Finished at: 2019-09-05T09:40:35Z
[INFO] ------------------------------------------------------------------------
[ERROR] No goals have been specified for this build. You must specify a valid lifecycle phase or a goal in the format <plugin-prefix>:<goal> or <plugin-group-id>:<plugin-artifact-id>[:<plugin-version>]:<goal>. Available lifecycle phases are: validate, initialize, generate-sources, process-sources, generate-resources, process-resources, compile, process-classes, generate-test-sources, process-test-sources, generate-test-resources, process-test-resources, test-compile, process-test-classes, test, prepare-package, package, pre-integration-test, integration-test, post-integration-test, verify, i

In [0]:
!pyspark --packages databricks:tensorframes:0.6.0-s_2.11

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
Type "help", "copyright", "credits" or "license" for more information.
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/content/spark-2.4.3-bin-hadoop2.6/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
databricks#tensorframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6dd6649c-e2da-4c8a-8159-a411d07a5b1a;1.0
	confs: [default]
	found databricks#tensorframes;0.6.0-s_2.11 in spark-packages
	found org.apache.commons#commons-proxy;1.0 in central
	found org.apache.commons#commons-lang3;3.4 in central
	found com.typesafe.scala-logging#scala-logging-api_2.11;2.1.2 in central
	found com.typesafe.scala-logging#scala-logging-slf4j_2.11;2.1.2 in central
	found org.scala-lang#scala-reflect;2.11.0 in central
	found org.slf4j#slf4j-api;1.7